## Primary Goal: take a file from a specific path and move it to a new directory

### Step 1: Get the Pre-Registered Names and associate it to the Source Rack position

In [ ]:
# Import block
import os
import csv
import shutil
import errno
import pandas as pd

In [ ]:
# Run this Hard-coded block to generate sample files in Colab for testing purposes

# Create sample directories in Colab
path_analytical = '/content/Lab Stations/OmniTasker/Barcodes/'
try:
    os.makedirs(path_analytical)
except FileExistsError:
    # directory already exists
    pass

path_destination = '/content/Lab Stations/OmniTasker/Tared_Weights/'
try:
    os.makedirs(path_destination)
except FileExistsError:
    # directory already exists
    pass

path_weights = '/content/Lab Stations/OmniTasker/Weight_Results/'
try:
    os.makedirs(path_weights)
except FileExistsError:
    # directory already exists
    pass

# Mock data for a sample csv file from analytical team
data_analytical = [
    ["source rack position", "pre-registered molecule name", "molecular formula", "molecular weight", "exact mass", "purity"],
    ["A1", "TEST-000001", "C48H62N8O9ClF", 949.51, 949.3660, "90%"],
    ["B1", "TEST-000002", "C20H30N2O5", 366.46, 366.2195, "91%"],
    ["C1", "TEST-000003", "C25H40N4O8S", 532.67, 532.2638, "92%"],
    ["D1", "TEST-000004", "C30H50N6O11", 654.75, 654.3629, "88%"],
    ["E1", "TEST-000005", "C35H58N8O7", 671.89, 671.4502, "89%"],
    ["F1", "TEST-000006", "C40H64N10O12", 873.01, 872.4763, "93%"],
    ["G1", "TEST-000007", "C15H24N2O3", 252.36, 252.1804, "87%"],
    ["H1", "TEST-000008", "C22H34N4O9S", 510.59, 510.2171, "85%"],
    ["A2", "TEST-000009", "C28H44N6O13", 628.68, 628.3197, "84%"],
    ["B2", "TEST-000010", "C17H28N2O4", 300.41, 300.2115, "90%"],
    ["C2", "TEST-000011", "C19H32N4O6", 396.48, 396.2428, "92%"],
    ["D2", "TEST-000012", "C24H38N6O10S", 570.65, 570.2513, "91%"],
    ["E2", "TEST-000013", "C29H46N8O8", 586.72, 586.3482, "86%"],
    ["F2", "TEST-000014", "C34H54N10O9", 698.85, 698.4071, "88%"],
    ["G2", "TEST-000015", "C39H60N12O10", 810.95, 810.4569, "90%"],
    ["H2", "TEST-000016", "C21H34N4O7S2", 494.65, 494.1976, "87%"],
    ["A3", "TEST-000017", "C26H42N6O8S", 578.72, 578.2949, "85%"],
    ["B3", "TEST-000018", "C31H52N8O9", 656.80, 656.3895, "89%"],
    ["C3", "TEST-000019", "C36H58N10O10", 738.91, 738.4357, "93%"],
    ["D3", "TEST-000020", "C41H66N12O11", 870.03, 869.4972, "92%"],
]

file_analytical = 'test_csv_analytical.csv'

# Writing to the CSV file
with open(path_analytical + file_analytical, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data_analytical)

print(f"CSV file '{file_analytical}' created successfully at .")

# Mock data for some sample weight results produced from OmniTasker
data_weights = [
    ["source rack position", "weight (mg)"],
    ["E2", 5.87],
    ["A1", 6.42],
    ["C2", 5.96],
    ["B3", 6.55],
    ["D1", 6.12],
    ["F1", 7.03],
    ["H2", 5.78],
    ["B1", 6.89],
    ["A3", 6.20],
    ["G1", 4.95],
    ["D3", 6.60],
    ["F2", 6.38],
    ["H1", 6.75],
    ["C1", 5.67],
    ["G2", 7.22],
    ["E1", 6.01],
    ["A2", 5.88],
    ["B2", 6.33],
    ["C3", 6.44],
    ["D2", 5.99],
]

file_weights = 'test_weights.csv'

# Writing to the CSV file
with open(path_weights + file_weights, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data_weights)

print(f"CSV file '{file_weights}' created successfully at .")


CSV file 'test_csv_analytical.csv' created successfully at .
CSV file 'test_weights.csv' created successfully at .


In [ ]:
# Define the source and destination file paths
source_path = '/content/Lab Stations/OmniTasker/Barcodes/'
destination_path = '/content/Lab Stations/OmniTasker/Tared_Weights/'

# Specify file name
file_name = 'test_csv_analytical.csv'

# Check if file exists
file_path = source_path + file_name

if not os.path.isfile(file_path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), file_name)
else:
    df_analytical = pd.read_csv(file_path)

In [ ]:
# Preview of first five rows in csv analytical
df_analytical.head(5)

,source rack position,pre-registered molecule name,molecular formula,molecular weight,exact mass,purity
0,A1,TEST-000001,C48H62N8O9ClF,949.51,949.3660,90%
1,B1,TEST-000002,C20H30N2O5,366.46,366.2195,91%
2,C1,TEST-000003,C25H40N4O8S,532.67,532.2638,92%
3,D1,TEST-000004,C30H50N6O11,654.75,654.3629,88%
4,E1,TEST-000005,C35H58N8O7,671.89,671.4502,89%


### Step 2: OmniTasker weighs out all the compounds and put back in source rack position
**Assumptions**
- Output is a CSV file with two columns: source rack position, weight (mg)
- The order of the weighing process is random (i.e. need to re-match rows based on source rack position)
- Could be false: each rack position is weighed only once (i.e. no rows with duplicate source rack position)
- The file will be in a different directory (For testing purposes: "Weight_Results")

In [ ]:
# Define the OmniTasker weight file path
weight_path = '/content/Lab Stations/OmniTasker/Weight_Results/'

# Specify weight file name
result_name = 'test_weights.csv'

result_path = weight_path + result_name

if not os.path.isfile(result_path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), result_name)
else:
    df_weight = pd.read_csv(result_path)

In [ ]:
# Preview of first five rows in weight results
df_weight.head(5)

,source rack position,weight (mg)
0,E2,5.87
1,A1,6.42
2,C2,5.96
3,B3,6.55
4,D1,6.12


In [ ]:
# Merge csv analytical with weight information based on source rack position
df_associated = df_analytical.merge(df_weight, how='left', on='source rack position')
df_associated

,source rack position,pre-registered molecule name,molecular formula,molecular weight,exact mass,purity,weight (mg)
0,A1,TEST-000001,C48H62N8O9ClF,949.51,949.3660,90%,6.42
1,B1,TEST-000002,C20H30N2O5,366.46,366.2195,91%,6.89
2,C1,TEST-000003,C25H40N4O8S,532.67,532.2638,92%,5.67
3,D1,TEST-000004,C30H50N6O11,654.75,654.3629,88%,6.12
4,E1,TEST-000005,C35H58N8O7,671.89,671.4502,89%,6.01
5,F1,TEST-000006,C40H64N10O12,873.01,872.4763,93%,7.03
6,G1,TEST-000007,C15H24N2O3,252.36,252.1804,87%,4.95
7,H1,TEST-000008,C22H34N4O9S,510.59,510.2171,85%,6.75
8,A2,TEST-000009,C28H44N6O13,628.68,628.3197,84%,5.88
9,B2,TEST-000010,C17H28N2O4,300.41,300.2115,90%,6.33


### Step 3: Weights need to be calculated for Reconstitution at 10mM DMSO

$\text{Volume (mL)} = \frac{\text{Mass}}{\text{Concentration } \times \text{ Molecular Weight}}$

- Molarity: millimolar  ($10^{-3}$ mol/L)
- Molecular Weight: g/mol
- Mass: mg
- Volume: mL


In [ ]:
# DMSO amount calculation function with unit conversion included
def dmso_calculation(mw: float, mass: float, mm = 10) -> float:
    return mass/(1e-6*mm*(mw*1e3))

#### Path 1 (if weight < 6mg): weight is calculated to know how much DMSO to add to weight for 10mM


In [ ]:
# Row-wise operations to calculate for any rows that has weight less than 6mg
df_path1 = df_associated[df_associated['weight (mg)'] < 6.00]
dmso_vol = df_path1.apply(lambda x: dmso_calculation(x['molecular weight'], x['weight (mg)']), axis=1)
dmso_vol

2     1.064449
6     1.961484
8     0.935293
10    1.503228
11    1.049680
12    1.000477
15    1.168503
dtype: float64

In [ ]:
# Merge the results with dataframe (NaN results for weight over 6mg)
df_associated['Amount of DMSO added for 10mM Stock'] = dmso_vol
df_associated

,source rack position,pre-registered molecule name,molecular formula,molecular weight,exact mass,purity,weight (mg),Amount of DMSO added for 10mM Stock
0,A1,TEST-000001,C48H62N8O9ClF,949.51,949.3660,90%,6.42,NaN
1,B1,TEST-000002,C20H30N2O5,366.46,366.2195,91%,6.89,NaN
2,C1,TEST-000003,C25H40N4O8S,532.67,532.2638,92%,5.67,1.064449
3,D1,TEST-000004,C30H50N6O11,654.75,654.3629,88%,6.12,NaN
4,E1,TEST-000005,C35H58N8O7,671.89,671.4502,89%,6.01,NaN
5,F1,TEST-000006,C40H64N10O12,873.01,872.4763,93%,7.03,NaN
6,G1,TEST-000007,C15H24N2O3,252.36,252.1804,87%,4.95,1.961484
7,H1,TEST-000008,C22H34N4O9S,510.59,510.2171,85%,6.75,NaN
8,A2,TEST-000009,C28H44N6O13,628.68,628.3197,84%,5.88,0.935293
9,B2,TEST-000010,C17H28N2O4,300.41,300.2115,90%,6.33,NaN


#### Path 2 (if weight > 6mg):
1. Stock amount of MeOH:CD2Cl2 is added to each vial
2. Solution is then mixed to make sure everything is dissolved
3. Source Vial is then split into 2 or more different barcoded intermediate scintillation vials
	- Barcoded Scintillation vial needs to be captured associated with Source Rack position
	- The quantity that is split needs to also be recorded
4. Scintillation vials get dried down
	- Recorded weight of new vial is recorded
	- Recorded weight gets associated to barcoded vial intermediate
5. Vial Barcode Intermediate gets pushed to Path 1



In [ ]:
# WIP

### Final: save and upload the file to the destination path

In [ ]:
# Output the resulting dataframe to csv at the destination path
try:
    df_associated.to_csv(destination_path + file_name, index=False)
except Exception as e:
    print(f"Error: {e}")